# Campeonato Brasileiro

Esse projeto usa dados fictícios, encontrados na plataforma *Kaggle*: https://www.kaggle.com/datasets/adaoduque/campeonato-brasileiro-de-futebol

### Importando os dados

In [2]:
import gdown

url = "https://drive.google.com/file/d/1hBe8Huudf6MhIpnZYlSqsvzJLScgZzCO/view?usp=sharing"
output = 'brasileirao.zip'

gdown.download(url, output, quiet=False)

C:\Users\guilh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\gdown\parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1hBe8Huudf6MhIpnZYlSqsvzJLScgZzCO
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1hBe8Huudf6MhIpnZYlSqsvzJLScgZzCO/view?usp=sharing
To: d:\Users\guilh\Documents\Python\campeonato-brasileiro\brasileirao.zip
70.4kB [00:00, 23.5MB/s]


'brasileirao.zip'

In [ ]:
from zipfile import ZipFile
import os
  
with ZipFile("brasileirao.zip", 'r') as zObject:
    zObject.extractall(path=os.getcwd()+"/data")

### Tratamento dos dados

#### Dataframes

Importando os dataframes

In [67]:
import pandas as pd

cartoes = pd.read_csv(os.getcwd() + '\data\campeonato-brasileiro-cartoes.csv')
gols = pd.read_csv(os.getcwd() + '\data\campeonato-brasileiro-gols.csv')
estatisticas = pd.read_csv(os.getcwd() + '\data\campeonato-brasileiro-estatisticas-full.csv')
estadios = pd.read_csv(os.getcwd() + '\data\estadios.csv')
df = pd.read_csv(os.getcwd() + '\data\campeonato-brasileiro-full.csv')

Visualizando cada um dos dataframes:

In [7]:
df.head()

,ID,rodata,data,hora,mandante,visitante,formacao_mandante,formacao_visitante,tecnico_mandante,tecnico_visitante,vencedor,arena,mandante_Placar,visitante_Placar,mandante_Estado,visitante_Estado
0,1,1,29/3/2003,16:00,Guarani,Vasco,NaN,NaN,NaN,NaN,Guarani,Brinco de Ouro,4,2,SP,RJ
1,2,1,29/3/2003,16:00,Athletico-PR,Gremio,NaN,NaN,NaN,NaN,Athletico-PR,Arena da Baixada,2,0,PR,RS
2,3,1,30/3/2003,16:00,Flamengo,Coritiba,NaN,NaN,NaN,NaN,-,Maracanã,1,1,RJ,PR
3,4,1,30/3/2003,16:00,Goias,Paysandu,NaN,NaN,NaN,NaN,-,Serra Dourada,2,2,GO,PA
4,5,1,30/3/2003,16:00,Internacional,Ponte Preta,NaN,NaN,NaN,NaN,-,Beira Rio,1,1,RS,SP


#### Feature Engineering

In [68]:
df["pontos_mandante"] = 0

for i, row in df.iterrows():
    if(row['mandante'] == row['vencedor']): df.loc[i,'pontos_mandante'] = 3
    elif(row['vencedor'] == '-'): df.loc[i,'pontos_mandante'] = 1
    else: df.loc[i,'pontos_mandante'] = 0

anos = range(2002, 2022)

df['ano'] = pd.DatetimeIndex(df['data']).year

C:\Users\guilh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\indexes\datetimes.py:327: UserWarning: Parsing '15/10/2003' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  dtarr = DatetimeArray._from_sequence_not_strict(
C:\Users\guilh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\indexes\datetimes.py:327: UserWarning: Parsing '18/10/2003' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  dtarr = DatetimeArray._from_sequence_not_strict(
C:\Users\guilh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\indexes\datetimes.py:327: UserWarning: Parsing '19/10/2003' in DD/MM/YYYY format. Provide format or specify infer_datetime_form

In [70]:
estadios.head()

,Time,Estádio
0,Guarani,Brinco de Ouro
1,Athletico-PR,Arena da Baixada
2,Athletico-PR,Estádio Joaquim Américo Guimarães
3,Flamengo,Maracanã
4,Goias,Serra Dourada


In [100]:
resultados = pd.DataFrame()

for ano in anos:
    for i, row in estadios.iterrows():
        pontos_feitos = sum((df["pontos_mandante"] != 0) & (df["arena"] == row['Estádio']) & (df["ano"] == ano))
        pontos_totais = sum((df["arena"] == row['Estádio']) & (df["ano"] == ano))*3

        pontos_totais_possiveis = sum((df["mandante"] == row['Time']) & (df["ano"] == ano))*3 + sum((df["visitante"] == row['Time']) & (df["ano"] == ano))*3
        pontos_feitos_possiveis = (
                                   sum((df["vencedor"] == row['Time']) & (df["ano"] == ano))*3 +
                                   sum((df["visitante"] == row['Time']) & (df["ano"] == ano) & (df["vencedor"] == "-")) +
                                   sum((df["mandante"] == row['Time']) & (df["ano"] == ano) & (df["vencedor"] == "-"))
                                  )

        try:
            p = (pontos_feitos/pontos_totais)*(pontos_feitos_possiveis/pontos_totais_possiveis)
        except ZeroDivisionError:
            p = "-"

        resultados.loc[row['Estádio'], ano] = p

In [103]:
resultados.head()
resultados.loc["Vila Belmiro", :]

2002           -
2003    0.192633
2004    0.177039
2005     0.12854
2006    0.165119
2007    0.143121
2008    0.103878
2009    0.101135
2010    0.133041
2011    0.113645
2012    0.145855
2013    0.142857
2014    0.107287
2015    0.160665
2016    0.179922
2017      0.1417
2018    0.121832
2019    0.204353
2020    0.137566
2021    0.116686
Name: Vila Belmiro, dtype: object

In [ ]:
for i, row in estadios.iterrows():
    resultados.loc[row['Estádio'], :]

In [99]:
import matplotlib.pyplot as plt

num=0
for column in resultados:
    num+=1
        
    for v in range(2002, 2022):
        for i, row in estadios.iterrows():
            plt.plot(resultados[row['Estádio']], resultados[v], marker='', color='grey', linewidth=0.6, alpha=0.3)
 
# general title
plt.suptitle("Evolução de gols no Brasileirão por time.  2000 - 2019", fontsize=13, fontweight=0, color='black', style='italic', y=1.02)
 
# Axis title
plt.text(0.5, 0.02, 'Time', ha='center', va='center')
plt.text(0.06, 0.5, 'Note', ha='center', va='center', rotation='vertical')

KeyError: 'Brinco de Ouro'

Mais da metade das partidas não tem os técnicos dos times, nem a formação usada

In [12]:
# Ano a ano, qual foi o aproveitamento do time no estádio (só times em casa)
# - Multiplicar o aproveitamento do time pela razão entre a quantidade de pontos feitas em casa/total
# - Ver o "peso" do estádio
df['arena'].unique()

for i in df['arena'].unique():
    pontos = sum((df['arena'] == i) | (df['arena'] == i))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="ticks", color_codes=True)
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['figure.dpi'] = 150

In [ ]:
cp = pd.DataFrame({'Ano': [year for year in range(2003, 2021)],
                  'Partidas': 0,
                  'Vitórias do mandante': 0,
                  'Vitórias do visitante': 0,
                  'Empates': 0,
                  'Gols do Mandante': 0,
                  'Gols do Visitante': 0})
cp = cp.set_index('Ano')

for index, row in df.iterrows():
    year = row['Data'].year
    if year == 2021:
        year = 2020
    home_goal = row['Mandante Placar']
    away_goal = row['Visitante Placar']
    cp.loc[year]['Partidas'] += 1
    cp.loc[year]['Gols do Mandante'] += home_goal
    cp.loc[year]['Gols do Visitante'] += away_goal
    if home_goal == away_goal:
        cp.loc[year]['Empates'] += 1
    elif home_goal > away_goal:
        cp.loc[year]['Vitórias do mandante'] += 1
    else:
        cp.loc[year]['Vitórias do visitante'] += 1
        
cp

In [ ]:
# Qual time mais teve posse de bola
# Técnico mais vitorias em casa
# Técnico com mais vitorias fora de casa
# Jogador que mais fez mais de um gol por partida

# Qual foi o técnico mais impactante: Ele estar em campo a chance de vitória era maior
# Qual foi o jogador mais impactante: Ele estar em campo a chance de vitória era maior

# Qual foi o jogador mais impactante: Ele estar em campo a posse de bola é maior
# Qual foi o técnico mais impactante: Ele estar em campo a posse de bola é maior

# Existe alguma relação entre posse de bola e vitória?
# Existe alguma relação entre posse de bola e gols marcados?
# Existe alguma relação entre posse de bola e gols sofridos?

# Qual é o time que mais ganhos em casa x derrotas? (Ano a Ano (%) por time)

# Kmeans para achar as melhores partidas do brasileirão (Santos x Flamengo - Aflitos - Corinthians x Palmeiras)

# Qual é o melhor clássico em relação a vitoria, derrota, empate, gols, posse e público?

# Qual maior média de público entre dois times?

# Média de público x tempo por time: Qual time mais cresceu? -> Identificar variação na derivada da curva pra times com novos estádios (Palmeiras)